In [1]:
import pandas as pd
import numpy
import codecs
import csv
import json
import pprint
import re

filename = "math sample for API testing.csv"

fields ={'Internal ID': 'Internal_ID',
         'Strand': 'Strand',
         'Sub-Strand': 'Substrand',
         'Content Focus': 'Content_Focus',
         'Component': 'Component',
         'Number': 'Number',
         'Digits': 'Digits',
         'Shape': 'Shape',
         'Units': 'Units',
         'Steps': 'Steps',
         'Context': 'Context',
         'SIG Stem': 'SIG_Stem',
         'SIG Options': 'SIG_Options',
         'Additional Modifiers': 'Additional_Modifiers'}

In [2]:
'''
Data structure

[list of internal IDs]
    {Organizers: 
        {Strand: Strand1}
        {Strand_GUID: GUID}
            {Substrand: Substrand1}
            {Substrand_GUID: GUID}
                {Content_Focus: Content_Focus1}
                {Content_Focus_GUID: GUID}
                    {Component: Component}
                    {Component_GUID: GUID}}
    {Descriptors:
        {Number: [N1, N2, ...]}
            {Digits: [D1, D2, ...]}
        {Shape: [Shape1, ...]}
        {Units: [Units1, ...]}
        {Steps: Steps}
        {Context: Context}
        {SIG Stem: [Sig Stem1, ...]}
        {SIG Options: [SIG Options1, ...]}
        {Additional Modifiers: [AM1, ...]}
'''

'\nData structure\n\n[list of internal IDs]\n    {Organizers: \n        {Strand: Strand1}\n        {Strand_GUID: GUID}\n            {Substrand: Substrand1}\n            {Substrand_GUID: GUID}\n                {Content_Focus: Content_Focus1}\n                {Content_Focus_GUID: GUID}\n                    {Component: Component}\n                    {Component_GUID: GUID}}\n    {Descriptors:\n        {Number: [N1, N2, ...]}\n            {Digits: [D1, D2, ...]}\n        {Shape: [Shape1, ...]}\n        {Units: [Units1, ...]}\n        {Steps: Steps}\n        {Context: Context}\n        {SIG Stem: [Sig Stem1, ...]}\n        {SIG Options: [SIG Options1, ...]}\n        {Additional Modifiers: [AM1, ...]}\n'

In [3]:
def process_file(filename, fields):
    process_fields = fields.keys()
    data = []
    holding = []
    data_keys = ['Internal ID', 'Strand','Substrand', 'Content_focus', 'Component', 'Number', 'Digits', 'Shape', 'Units', 'Steps', 'Context', 'SIG Stem', 'SIG Options', 'Additional Modifiers']
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for i in range(3):
            l = reader.next()

        for line in reader:
            holding_test = 'FALSE'
            row = {}
            for key,value in fields.items():
                row[value] = (line[key]).split("; ")
                #If strand, substrand, Content Focus or Numbers field in raw data has ";", then add to holding list
                if (value == "Strand" or value == "Substrand" or value == "Content_Focus" or value == "Number") and (re.search(';', line[key])):
                    holding_test = 'TRUE'
            if holding_test == 'TRUE':
                holding.append(row) 
            else:
                data.append(row)
    #Process holding list
    #Do the row have multiple values in strand both and substrand? If yes, then consult table_of_values to figure out what should get 
    #paired together
    pprint.pprint(holding)
    return data

In [4]:
data = process_file(filename, fields)

[{'Additional_Modifiers': ['N/A'],
  'Component': ['N/A'],
  'Content_Focus': ['Use unit circle', 'Know trigonometric ratios'],
  'Context': ['No'],
  'Digits': ['Denominator: 2', 'Denominator: 3'],
  'Internal_ID': ['333009'],
  'Number': ['Irrational numbers',
             'Negative and positive fractions',
             'Square roots'],
  'SIG_Options': ['N/A'],
  'SIG_Stem': ['N/A'],
  'Shape': ['Angles', 'Circle', 'Line', 'Point', 'Right triangle', 'Arc'],
  'Steps': ['two Step'],
  'Strand': ['Geometry and Measurement', 'Algebra'],
  'Substrand': ['Shapes', 'Functions'],
  'Units': ['N/A']},
 {'Additional_Modifiers': ['N/A'],
  'Component': ['Arc length'],
  'Content_Focus': ['Use unit circle', 'Use proportionality of circles'],
  'Context': ['No'],
  'Digits': ['Ones',
             'Denominator: 2',
             'Denominator: 3',
             'Denominator: 4',
             'Denominator: 6'],
  'Internal_ID': ['331547'],
  'Number': ['Irrational numbers',
             'Integers',


In [5]:
pprint.pprint(data)

[{'Additional_Modifiers': ['N/A'],
  'Component': ['N/A'],
  'Content_Focus': ['Use trigonometric ratios'],
  'Context': ['No'],
  'Digits': ['Denominator: 5'],
  'Internal_ID': ['331526'],
  'Number': ['Fractions'],
  'SIG_Options': ['N/A'],
  'SIG_Stem': ['Diagram'],
  'Shape': ['Angles', 'Composite figure', 'Right triangle'],
  'Steps': ['N/A'],
  'Strand': ['Geometry and Measurement'],
  'Substrand': ['Shapes'],
  'Units': ['N/A']}]


In [6]:
#Make every internal ID of int data type "Internal ID"

In [7]:
#load GUID file

In [8]:
#problem to solve for looking up GUID: when strand/substrand gives context to component, 
#if semi-colon separated, then will have to allow for non-matches

In [9]:
def Load_GUIDs(filename):
    data = []
    fields = {}
    with open(filename, "r") as f:
        reader = csv.DictReader(f)
        for line in reader:
            data.append(line)
    #strip capitalization for matching purposes
    data = pd.DataFrame(data)
    lowercase = lambda x: x.lower()
    data['Title'] = data['Title'].apply(lowercase)
    return data

In [10]:
steps_df = Load_GUIDs("Math_Steps.csv")
shapes_df = Load_GUIDs("Math_Shapes.csv")
numbers_df = Load_GUIDs("Math_Number.csv")
units_df = Load_GUIDs("Math_Units_Modified.csv")

In [11]:
print units_df

        Subject         Title                             Unique ID
0   Mathematics         money  7C0657E6-97C4-11E6-B4CA-B3D929C466BA
1   Mathematics       degrees  7C076EC4-97C4-11E6-B4CA-B3D929C466BA
2   Mathematics       derived  7C085C9E-97C4-11E6-B4CA-B3D929C466BA
3   Mathematics          time  7C09437A-97C4-11E6-B4CA-B3D929C466BA
4   Mathematics        metric  7C0A2DBC-97C4-11E6-B4CA-B3D929C466BA
5   Mathematics       generic  7C0B3838-97C4-11E6-B4CA-B3D929C466BA
6   Mathematics     customary  7C0C3152-97C4-11E6-B4CA-B3D929C466BA
7   Mathematics  non-standard  7C0D22C4-97C4-11E6-B4CA-B3D929C466BA
8   Mathematics       radians  7C0E0CB6-97C4-11E6-B4CA-B3D929C466BA
9   Mathematics    fahrenheit  F1A8F300-A606-11E6-967A-7DB129C466BA
10  Mathematics       celsius  F1AA027C-A606-11E6-967A-7DB129C466BA


In [12]:
def get_GUID_lite(tag, df): 
    #get GUID for steps, number, units, shape
    tag = tag.lower
    matches = len(df.ix[df['Title'] == tag])
    if matches == 1:
        GUID = df.ix[df['Title'] == tag, 'Unique ID'].item()
    elif matches ==0:
        GUID = "no matches found"
    else: GUID = "resolve multiple values"

    return GUID

In [ ]:

#create version for content focus and component
#Play around with passing multiple conditions into first .ix argument


In [ ]:
test = get_GUID_lite('Decimals to fractions', numbers_df)
print test

In [ ]:
def lowercase(text):
    return text.lower()

#Adjusted to just apply lowercase function since special characters not used and using Python 2.7 not 3.0

In [ ]:
import unicodedata
test = "BuTt"
print test.lower()